# Análise dos dados das variáveis relacionadas ao IDEB

O notebook contém experimentos com essas variáveis. São utilizadas duas bases de dados:<br>
- Váriáveis derivadas do SAEB (informações socioeconômicas)<br> 
- Variáveis com informações das escolas e professores<br>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, RFE, RFECV, SelectFromModel
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy import array 

## Importando bases de dados

In [ ]:
#Importando variáveis derivadas do SAEB

#base_estudantes_se = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_se.csv')

base_estudantes_se = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_se.csv')

#base_estudantes_se.set_index('ID_ESCOLA', inplace=True)

#base_estudantes_al = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_al.csv')

base_estudantes_al = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_al.csv')

#base_estudantes_al.set_index('ID_ESCOLA', inplace=True)

base_estudantes_ba = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_ba.csv')

#base_estudantes_ba.set_index('ID_ESCOLA', inplace=True)

dados_pessoais = pd.merge(base_estudantes_se, base_estudantes_al, how = 'outer')

dados_pessoais = pd.merge(base_estudantes_ba, dados_pessoais, how = 'outer')

In [ ]:
dados_pessoais.set_index('ID_ESCOLA', inplace=True)

In [ ]:
dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29211590,2017,2,29,2900108,2,2,1,1.0,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,...,6,1,8,16,29,17,1,6,23,14,5,13,4,7,8,21,10,8,8,2,10,0,8,25,15,8,4,1,8,9,5,10,3,7,5,5,3,6,6,7
29032539,2017,2,29,2900207,2,2,1,1.0,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,...,13,1,15,27,30,32,42,9,44,18,10,27,9,29,25,33,12,9,17,4,45,0,11,49,34,19,13,7,12,16,5,14,17,13,11,15,10,11,14,13
29033047,2017,2,29,2900207,2,2,2,1.0,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,...,3,2,17,19,25,24,14,11,15,17,12,18,5,24,14,29,12,6,10,1,30,1,8,28,26,14,9,9,7,14,4,13,8,4,14,6,12,3,10,6
29446740,2017,2,29,2900306,2,2,1,1.0,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,...,6,0,9,22,26,24,19,1,20,12,14,33,9,12,8,17,17,11,20,1,25,0,4,31,32,19,9,3,3,10,4,12,9,13,7,10,10,8,7,4
61206519,2017,2,29,2900355,2,2,1,1.0,21,58,58,22,51,29,51,27,35,43,76,3,69,10,62,17,73,6,51,28,12,67,18,56,64,14,33,42,66,9,5,52,...,2,1,8,21,27,12,11,11,30,6,10,6,5,12,23,28,9,4,2,0,11,0,6,21,18,11,10,2,10,6,5,9,10,9,5,9,3,7,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61206951,2017,2,27,6307176,2,4,1,1.0,8,13,20,1,18,3,20,1,15,6,21,0,20,1,17,4,21,0,18,3,1,20,2,19,20,1,9,11,20,0,1,15,...,0,0,2,8,5,4,2,0,0,1,2,7,9,2,0,0,2,2,10,3,4,0,5,12,4,0,0,0,0,2,3,2,2,2,2,4,0,0,1,2
61178197,2017,2,27,6307192,2,4,1,1.0,7,23,30,0,23,7,29,1,24,5,29,1,27,3,24,6,30,0,21,9,0,29,1,29,20,10,8,21,28,2,0,28,...,0,0,2,8,15,5,0,0,0,0,3,13,12,2,0,0,0,5,17,7,1,0,5,18,5,2,0,0,0,1,3,3,2,2,4,3,1,1,4,3
61197558,2017,2,27,6307192,2,4,1,1.0,6,6,12,0,10,2,11,0,8,3,12,0,11,1,11,1,12,0,9,3,0,12,1,11,7,5,12,0,10,0,2,7,...,1,0,1,2,5,4,0,1,0,0,1,8,1,1,0,0,0,1,9,0,2,0,1,7,3,1,0,0,0,0,1,0,1,2,0,3,0,1,1,2


**Pré-processamento da base de dados pessoais**

In [ ]:
#Função que verifica se o ID de uma escola é inválido
#Segundo o Dicionário de Dados da base o SAEB 6 - Os códigos dos Municípios e 
#das Escolas que começam com o dígito "6" são máscaras, isto é, são códigos fictícios.
def retira_id_ficticio(id_escola):
    if id_escola[0] == "6":
        return True
    else:
        return False

In [ ]:
#Se retirou as escolas fictícias
for escola in dados_pessoais.index:
    if (retira_id_ficticio(str(escola))):
        dados_pessoais.drop(escola, inplace=True)
        

dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29211590,2017,2,29,2900108,2,2,1,1.0,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,...,6,1,8,16,29,17,1,6,23,14,5,13,4,7,8,21,10,8,8,2,10,0,8,25,15,8,4,1,8,9,5,10,3,7,5,5,3,6,6,7
29032539,2017,2,29,2900207,2,2,1,1.0,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,...,13,1,15,27,30,32,42,9,44,18,10,27,9,29,25,33,12,9,17,4,45,0,11,49,34,19,13,7,12,16,5,14,17,13,11,15,10,11,14,13
29033047,2017,2,29,2900207,2,2,2,1.0,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,...,3,2,17,19,25,24,14,11,15,17,12,18,5,24,14,29,12,6,10,1,30,1,8,28,26,14,9,9,7,14,4,13,8,4,14,6,12,3,10,6
29446740,2017,2,29,2900306,2,2,1,1.0,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,...,6,0,9,22,26,24,19,1,20,12,14,33,9,12,8,17,17,11,20,1,25,0,4,31,32,19,9,3,3,10,4,12,9,13,7,10,10,8,7,4
29236606,2017,2,29,2900504,2,2,1,1.0,33,64,85,17,76,26,66,34,47,53,100,2,88,13,82,19,99,3,79,23,16,84,26,74,74,27,48,54,86,11,4,68,...,2,0,5,24,25,19,27,8,29,15,7,19,4,20,14,30,14,1,8,1,34,0,11,29,28,16,11,5,2,7,10,9,4,8,9,8,10,10,14,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27024059,2017,2,27,2709301,2,2,1,1.0,73,106,134,49,117,66,114,67,84,93,180,3,171,11,152,29,178,4,110,72,27,154,49,128,154,23,127,53,140,29,10,108,...,7,2,16,43,54,32,37,23,47,33,24,24,10,20,32,36,25,19,19,2,49,0,5,48,54,34,14,10,18,13,12,16,14,19,19,17,12,15,19,16
27024067,2017,2,27,2709301,2,2,1,1.0,21,17,30,9,28,11,24,16,16,23,40,0,39,1,37,3,40,0,29,11,6,32,10,30,31,9,28,12,35,4,0,26,...,3,1,5,8,10,9,6,5,13,4,5,2,2,8,6,12,7,4,2,0,9,1,3,11,12,6,3,3,0,5,1,4,4,1,3,1,4,1,6,5
27025047,2017,2,27,2709301,2,2,1,1.0,31,59,82,9,65,26,77,15,52,38,89,3,90,2,83,9,90,2,72,20,10,80,5,85,87,3,53,37,77,9,4,59,...,1,0,8,16,40,18,9,5,15,16,15,26,12,2,4,22,18,15,13,3,17,0,5,46,25,10,2,0,3,11,3,8,6,5,7,8,12,7,10,6


In [ ]:
#Importando variáveis com informações das escolas e professores
#base_escola_se = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-SE.csv')

base_escola_se = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-SE.csv')

#base_escola_se.set_index('CO_ENTIDADE', inplace=True)

#base_escola_al = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-AL.csv')

base_escola_al = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-AL.csv')

#base_escola_al.set_index('CO_ENTIDADE', inplace=True)

#base_escola_ba = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-BA.csv')

base_escola_ba = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-BA.csv')

base_escola_ba.rename(columns={'Unnamed: 0': 'CO_ENTIDADE'}, inplace=True)

#base_escola_ba.set_index('CO_ENTIDADE', inplace=True)

dados_escola = pd.merge(base_escola_se, base_escola_al, how = 'outer')

In [ ]:
dados_escola = pd.concat([dados_escola, base_escola_ba], join='outer')

dados_escola.set_index('CO_ENTIDADE', inplace=True)

In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesDivSocCult,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf,Ideb-2017,QtdeDocentesDivSocCulta,asQtdeDocentesEfetivos,QtdedocentesCLT
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28017943,2017,COLEGIO AMADEUS LTDA,0.0,1,16/01/2017,24/11/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0,0.0,16,3,0,27,1,45.0,0.0,0.0,0.0,0.0,0.0,45.0,45,345,45,7.0,6.9,NaN,NaN,NaN
28010124,2017,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,2.0,1,01/02/2017,08/12/2017,2,2802,28005,28,2807402,280740205,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0,0.0,5,0,0,19,3,24.0,0.0,0.0,0.0,0.0,0.0,24.0,24,103,24,4.0,6.6,NaN,NaN,NaN
28018052,2017,COLEGIO SANTANNA DE SERGIPE LTDA,0.0,1,09/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0,0.0,5,0,0,12,1,17.0,0.0,0.0,0.0,0.0,0.0,17.0,17,89,17,5.0,6.4,NaN,NaN,NaN
28032497,2017,COLEGIO NOTA DEZ,3.0,1,23/01/2017,15/12/2017,2,2802,28004,28,2803906,280390605,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0,0.0,2,1,0,5,0,11.0,0.0,0.0,0.0,0.0,0.0,11.0,11,21,11,1.0,6.1,NaN,NaN,NaN
28017773,2017,COL AMERICANO BATISTA,0.0,1,16/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0,0.0,6,1,0,15,0,22.0,0.0,0.0,0.0,0.0,0.0,22.0,22,152,22,6.0,5.8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223687,2017,EE - COLEGIO ESTADUAL EDIVALDO BOAVENTU

In [ ]:
col_ideb = dados_escola['Ideb-2017']

dados_escola.drop('Ideb-2017', axis=1, inplace=True)

In [ ]:
col_ideb

CO_ENTIDADE
28017943    6.9
28010124    6.6
28018052    6.4
28032497    6.1
28017773    5.8
           ... 
29223687    2.9
29035007    3.9
29242290    4.0
29237645    3.7
29337941    2.5
Name: Ideb-2017, Length: 670, dtype: float64

In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesDivSocCult,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf,QtdeDocentesDivSocCulta,asQtdeDocentesEfetivos,QtdedocentesCLT
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28017943,2017,COLEGIO AMADEUS LTDA,0.0,1,16/01/2017,24/11/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,0.0,16,3,0,27,1,45,0.0,0.0,0.0,0.0,0.0,45.0,45,345,45,7.0,NaN,NaN,NaN
28010124,2017,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,2.0,1,01/02/2017,08/12/2017,2,2802,28005,28,2807402,280740205,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,0,0.0,5,0,0,19,3,24,0.0,0.0,0.0,0.0,0.0,24.0,24,103,24,4.0,NaN,NaN,NaN
28018052,2017,COLEGIO SANTANNA DE SERGIPE LTDA,0.0,1,09/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0.0,5,0,0,12,1,17,0.0,0.0,0.0,0.0,0.0,17.0,17,89,17,5.0,NaN,NaN,NaN
28032497,2017,COLEGIO NOTA DEZ,3.0,1,23/01/2017,15/12/2017,2,2802,28004,28,2803906,280390605,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,2.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0.0,2,1,0,5,0,11,0.0,0.0,0.0,0.0,0.0,11.0,11,21,11,1.0,NaN,NaN,NaN
28017773,2017,COL AMERICANO BATISTA,0.0,1,16/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0.0,6,1,0,15,0,22,0.0,0.0,0.0,0.0,0.0,22.0,22,152,22,6.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223687,2017,EE - COLEGIO ESTADUAL EDIVALDO BOAVENTURA - PO...,9.0,1,06/02/2017,13/12/2017,2,2906,29024,29,2904308,290430805,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1.0,0,0,0,0,0,0,0,0.0,0,0,0.0,1,1,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,184,0,NaN,78,8,0,89,0,184,0.0,0.0,NaN,112.0,0.0,NaN,184,797,184,NaN,0.0,72.0,0.0
29035007,2017,EE - COLEGIO ESTADUAL POLIVAL

#### Pré-processamento da base das escolas

In [ ]:
#Retirando valores ausentes
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL               11
TP_CATEGORIA_ESCOLA_PRIVADA    194
IN_CONVENIADA_PP               194
TP_CONVENIO_PODER_PUBLICO      234
IN_MANT_ESCOLA_PRIVADA_EMP     194
                              ... 
QtdeDocentesCLT                574
RelAlunProf                    574
QtdeDocentesDivSocCulta         96
asQtdeDocentesEfetivos         125
QtdedocentesCLT                239
Length: 165, dtype: int64

In [ ]:
#Retirando colunas que apresentam mais de 60% do valores nulos
dados_escola.dropna(thresh=len(dados_escola)*0.6, axis=1, inplace=True)

#Preenchendo valores ausentes
dados_escola.fillna(0, inplace=True)

filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL              0
TP_CATEGORIA_ESCOLA_PRIVADA    0
IN_CONVENIADA_PP               0
TP_CONVENIO_PODER_PUBLICO      0
IN_MANT_ESCOLA_PRIVADA_EMP     0
                              ..
QtdeDocentesTempo              0
QtdeDocentesTerceir            0
QtdeDocentesDivSocCulta        0
asQtdeDocentesEfetivos         0
QtdedocentesCLT                0
Length: 161, dtype: int64

In [ ]:
dados_escola[listOfColumnNames] = dados_escola[listOfColumnNames].astype(int)

In [ ]:
dados_escola['Ideb-2017'] = col_ideb

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """Entry point for launching an IPython kernel.


In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,QtdeDocentesDivSocCulta,asQtdeDocentesEfetivos,QtdedocentesCLT,Ideb-2017
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28017943,2017,COLEGIO AMADEUS LTDA,0,1,16/01/2017,24/11/2017,2,2803,28011,28,2800308,280030805,4,1,1,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,16,3,0,27,1,45,0,0,0,0,45,345,45,0,0,0,6.9
28010124,2017,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,2,1,01/02/2017,08/12/2017,2,2802,28005,28,2807402,280740205,4,1,1,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,0,5,0,0,19,3,24,0,0,0,0,24,103,24,0,0,0,6.6
28018052,2017,COLEGIO SANTANNA DE SERGIPE LTDA,0,1,09/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,5,0,0,12,1,17,0,0,0,0,17,89,17,0,0,0,6.4
28032497,2017,COLEGIO NOTA DEZ,3,1,23/01/2017,15/12/2017,2,2802,28004,28,2803906,280390605,4,1,1,0,0,1,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,2,1,0,5,0,11,0,0,0,0,11,21,11,0,0,0,6.1
28017773,2017,COL AMERICANO BATISTA,0,1,16/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,6,1,0,15,0,22,0,0,0,0,22,152,22,0,0,0,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223687,2017,EE - COLEGIO ESTADUAL EDIVALDO BOAVENTURA - PO...,9,1,06/02/2017,13/12/2017,2,2906,29024,29,2904308,290430805,2,2,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,184,0,78,8,0,89,0,184,0,0,112,0,184,797,184,0,72,0,2.9
29035007,2017,EE - COLEGIO ESTADUAL POLIVALENTE DE PAULO AFONSO,24,1,06/02/2017,13/12/2017,2,2902,29005,29,2924009,292400905,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,289,0,282,6,0,1,0,332,0,1,30,0,333,1512,333,0,302,0,3.9
29242290,2017,EE - INSTITUTO DE EDUCACAO ANISIO TEIXEIRA,13,1,06/02/

In [ ]:
#Colunas categóricas
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.object]

listOfColumnNames = list(filteredColumns.index)

dados_escola[listOfColumnNames]

,NO_ENTIDADE,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO
CO_ENTIDADE,,,
28017943,COLEGIO AMADEUS LTDA,16/01/2017,24/11/2017
28010124,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,01/02/2017,08/12/2017
28018052,COLEGIO SANTANNA DE SERGIPE LTDA,09/01/2017,01/12/2017
28032497,COLEGIO NOTA DEZ,23/01/2017,15/12/2017
28017773,COL AMERICANO BATISTA,16/01/2017,01/12/2017
...,...,...,...
29223687,EE - COLEGIO ESTADUAL EDIVALDO BOAVENTURA - PO...,06/02/2017,13/12/2017
29035007,EE - COLEGIO ESTADUAL POLIVALENTE DE PAULO AFONSO,06/02/2017,13/12/2017
29242290,EE - INSTITUTO DE EDUCACAO ANISIO TEIXEIRA,06/02/2017,13/12/2017


In [ ]:
#Tratando datas, convertendo para timestamp

inicio = pd.to_datetime(dados_escola['DT_ANO_LETIVO_INICIO'], infer_datetime_format = True)
termino = pd.to_datetime(dados_escola['DT_ANO_LETIVO_TERMINO'], infer_datetime_format = True)

dados_escola['DT_ANO_LETIVO_INICIO'] = inicio.values.astype (np.int64)
dados_escola['DT_ANO_LETIVO_TERMINO'] = termino.values.astype (np.int64)

In [ ]:
dados_escola.drop('NO_ENTIDADE', axis=1, inplace=True)

In [ ]:
dados_escola.columns.to_list()

['NU_ANO_CENSO',
 'CO_ORGAO_REGIONAL',
 'TP_SITUACAO_FUNCIONAMENTO',
 'DT_ANO_LETIVO_INICIO',
 'DT_ANO_LETIVO_TERMINO',
 'CO_REGIAO',
 'CO_MESORREGIAO',
 'CO_MICRORREGIAO',
 'CO_UF',
 'CO_MUNICIPIO',
 'CO_DISTRITO',
 'TP_DEPENDENCIA',
 'TP_LOCALIZACAO',
 'TP_CATEGORIA_ESCOLA_PRIVADA',
 'IN_CONVENIADA_PP',
 'TP_CONVENIO_PODER_PUBLICO',
 'IN_MANT_ESCOLA_PRIVADA_EMP',
 'IN_MANT_ESCOLA_PRIVADA_ONG',
 'IN_MANT_ESCOLA_PRIVADA_SIND',
 'IN_MANT_ESCOLA_PRIVADA_SIST_S',
 'IN_MANT_ESCOLA_PRIVADA_S_FINS',
 'CO_ESCOLA_SEDE_VINCULADA',
 'CO_IES_OFERTANTE',
 'TP_REGULAMENTACAO',
 'IN_LOCAL_FUNC_PREDIO_ESCOLAR',
 'TP_OCUPACAO_PREDIO_ESCOLAR',
 'IN_LOCAL_FUNC_SALAS_EMPRESA',
 'IN_LOCAL_FUNC_SOCIOEDUCATIVO',
 'IN_LOCAL_FUNC_UNID_PRISIONAL',
 'IN_LOCAL_FUNC_PRISIONAL_SOCIO',
 'IN_LOCAL_FUNC_TEMPLO_IGREJA',
 'IN_LOCAL_FUNC_CASA_PROFESSOR',
 'IN_LOCAL_FUNC_GALPAO',
 'TP_OCUPACAO_GALPAO',
 'IN_LOCAL_FUNC_SALAS_OUTRA_ESC',
 'IN_LOCAL_FUNC_OUTROS',
 'IN_PREDIO_COMPARTILHADO',
 'IN_AGUA_FILTRADA',
 'IN_AGUA_RE

In [ ]:
dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29211590,2017,2,29,2900108,2,2,1,1.0,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,...,6,1,8,16,29,17,1,6,23,14,5,13,4,7,8,21,10,8,8,2,10,0,8,25,15,8,4,1,8,9,5,10,3,7,5,5,3,6,6,7
29032539,2017,2,29,2900207,2,2,1,1.0,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,...,13,1,15,27,30,32,42,9,44,18,10,27,9,29,25,33,12,9,17,4,45,0,11,49,34,19,13,7,12,16,5,14,17,13,11,15,10,11,14,13
29033047,2017,2,29,2900207,2,2,2,1.0,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,...,3,2,17,19,25,24,14,11,15,17,12,18,5,24,14,29,12,6,10,1,30,1,8,28,26,14,9,9,7,14,4,13,8,4,14,6,12,3,10,6
29446740,2017,2,29,2900306,2,2,1,1.0,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,...,6,0,9,22,26,24,19,1,20,12,14,33,9,12,8,17,17,11,20,1,25,0,4,31,32,19,9,3,3,10,4,12,9,13,7,10,10,8,7,4
29236606,2017,2,29,2900504,2,2,1,1.0,33,64,85,17,76,26,66,34,47,53,100,2,88,13,82,19,99,3,79,23,16,84,26,74,74,27,48,54,86,11,4,68,...,2,0,5,24,25,19,27,8,29,15,7,19,4,20,14,30,14,1,8,1,34,0,11,29,28,16,11,5,2,7,10,9,4,8,9,8,10,10,14,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27024059,2017,2,27,2709301,2,2,1,1.0,73,106,134,49,117,66,114,67,84,93,180,3,171,11,152,29,178,4,110,72,27,154,49,128,154,23,127,53,140,29,10,108,...,7,2,16,43,54,32,37,23,47,33,24,24,10,20,32,36,25,19,19,2,49,0,5,48,54,34,14,10,18,13,12,16,14,19,19,17,12,15,19,16
27024067,2017,2,27,2709301,2,2,1,1.0,21,17,30,9,28,11,24,16,16,23,40,0,39,1,37,3,40,0,29,11,6,32,10,30,31,9,28,12,35,4,0,26,...,3,1,5,8,10,9,6,5,13,4,5,2,2,8,6,12,7,4,2,0,9,1,3,11,12,6,3,3,0,5,1,4,4,1,3,1,4,1,6,5
27025047,2017,2,27,2709301,2,2,1,1.0,31,59,82,9,65,26,77,15,52,38,89,3,90,2,83,9,90,2,72,20,10,80,5,85,87,3,53,37,77,9,4,59,...,1,0,8,16,40,18,9,5,15,16,15,26,12,2,4,22,18,15,13,3,17,0,5,46,25,10,2,0,3,11,3,8,6,5,7,8,12,7,10,6


### Unindo as bases de dados 

Foi realizado join nos dois datasets, nesse momento se percebeu que não existem variáveis escolas particulares na base derivada do SAEB

In [ ]:
dataset = dados_pessoais.join(dados_escola, how="inner")

In [ ]:
dataset

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,QtdeDocentesDivSocCulta,asQtdeDocentesEfetivos,QtdedocentesCLT,Ideb-2017
27000095,2017,2,27,2700102,2,2,1,1.0,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,172,0,62,0,0,103,0,172,-1,-1,148,-1,172,508,172,0,24,-1,3.4
27001237,2017,2,27,2701605,2,2,1,1.0,76,108,144,44,130,58,113,68,77,100,177,6,167,16,162,21,179,4,133,49,43,139,55,119,146,36,102,75,153,30,4,123,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,250,0,111,0,0,89,31,250,-1,-1,225,14,250,792,250,0,11,-1,3.9
27002020,2017,2,27,2705002,2,2,1,1.0,41,60,78,21,70,28,66,29,49,47,95,2,88,9,89,8,91,5,73,24,13,80,36,52,84,8,61,24,79,20,1,64,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,222,0,100,0,0,86,0,222,-1,-1,217,-1,222,652,222,0,5,-1,3.3
27002063,2017,2,27,2705002,2,2,2,1.0,12,24,22,13,15,20,18,18,9,26,34,2,32,4,33,3,35,1,26,9,2,32,14,21,28,7,29,5,33,2,0,26,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63,0,20,0,0,43,0,73,-1,-1,40,20,73,309,73,0,13,-1,3.8
27002950,2017,2,27,2706422,2,2,1,1.0,70,72,109,31,88,52,89,46,57,77,134,3,123,13,115,22,133,4,98,38,14,122,36,102,93,46,83,52,116,22,3,81,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,117,0,72,7,0,36,0,121,-1,-1,84,-1,121,442,121,0,37,-1,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,2017,2,29,2923704,2,3,2,1.0,22,31,50,5,49,6,45,9,39,14,54,1,51,3,51,4,53,1,46,8,5,48,27,24,48,7,31,20,48,4,3,41,...,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,92,0,15,0,0,39,0,177,1,0,3,0,178,701,178,0,174,0,2.5
29848873,2017,2,29,2923100,2,2,2,1.0,8,18,18,8,17,9,15,9,9,15,25,0,24,0,22,3,25,0,17,8,1,24,9,13,21,3,15,9,25,0,0,17,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42,0,30,0,0,18,0,54,0,0,42,0,54,209,54,0,12,0,2.3
29886732,2017,2,29,2902252,2,2,2,1.0,2,4,5,0,4,1,4,1,3,2,5,0,5,0,4,0,4,0,5,0,0,4,1,3,5,0,1,2,5,1,0,2,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,93,0,56,8,0,127,0,199,0,0,193,0,199,389,199,0,6,0,1.8
29903726,2017,2,29,2933257,2,2,1,1.0,19,23,32,10,26,16,30,9,16,22,38,1,33,6,34,5,39,0,25,14,13,24,7,30,26,13,24,13,28,12,1,20,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42,0,38,0,0,19,0,63,0,0,47,0,63,194,63,0,16,0,3.5


In [ ]:
dados_escola.dtypes[dados_escola.dtypes == np.float64]

Ideb-2017    float64
dtype: object

In [ ]:
dataset['ID_DEPENDENCIA_ADM'].unique()

array([2, 1, 3])

In [ ]:
#dataset.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa Ideb/variaveis_derivadas_escola_estudante_4.0.csv')

### Discretização dos rótulos, nota do Ideb

**Discretização:** É o processo de transferência de funções contínuas, modelos, variáveis e equações em contrapartes discretas.

In [ ]:
# São três classes média, acima da média e abaixo da média
def discretizacao(ideb, mean_ideb):
    classe = []
    for nota in ideb['Ideb-2017']:
        if (mean_ideb == nota):
            classe.append('média')
        elif (mean_ideb < nota):
            classe.append('acima da média')
        else:
            classe.append('abaixo da média')
    return classe

In [ ]:
#discretizando as notas do Ideb

Ideb = dataset[['Ideb-2017']]

mean_Ideb = round(Ideb.mean(),1)

classe_ideb = discretizacao(Ideb, mean_Ideb['Ideb-2017'])

Ideb.insert(1, 'classe-Ideb', classe_ideb)

Ideb

,Ideb-2017,classe-Ideb
27000095,3.4,acima da média
27001237,3.9,acima da média
27002020,3.3,acima da média
27002063,3.8,acima da média
27002950,3.5,acima da média
...,...,...
29840635,2.5,abaixo da média
29848873,2.3,abaixo da média
29886732,1.8,abaixo da média
29903726,3.5,acima da média


In [ ]:
#Variavel a ser prevista
y = Ideb['classe-Ideb']

**Seleção de Atributos:**  É um passo da fase de pré-processamento que tem a finalidade de selecionar o melhor subconjunto de atributos que pode substituir o conjunto de dados original. 

https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/#:~:text=Embedded%20methods%20combine%20the%20qualities,penalization%20functions%20to%20reduce%20overfitting.

Abordagens:

- Filtro: a relevância dos atributos é avaliada independentemente – não
envolve o uso de um método de AM ;

- Wrapper (Envólucro): Avalia o subconjunto de atributos arti culado ao método de AM que realizará a indução do conceito;

- Embedded (Embutidos): Os métodos incorporados combinam as qualidades dos métodos de filtro e wrapper. Ele é implementado por algoritmos que possuem seus próprios métodos de seleção de recursos integrados.

In [ ]:
#Filter based
# f_classif: ANOVA F-value between label/feature for classification tasks.
#ANOVA stands for Analysis of variance. It is similar to LDA except for the fact that it is operated using one or more categorical 
#independent features and one continuous dependent feature. It provides a statistical test 
#of whether the means of several groups are equal or not.
def selecao_atributos(X, y, quant):
    select = SelectKBest(score_func=f_classif, k=quant)
    #select = SelectKBest(score_func=chi2, k=quant)
    z = select.fit_transform(X, y)
    
    filter = select.get_support(indices=True)
    features = array(X.columns)
    
    return X.iloc[:, filter]

In [ ]:
#Wrapper-based
from sklearn.cluster import KMeans

def wrapper_based(X, X_transform, y, quant):

  #select_features = RFE(estimator=KMeans(n_clusters=3), n_features_to_select=100)
  #select_features = RFE(estimator=SVC(kernel="linear"), n_features_to_select=100)
  select_features = RFECV(
      estimator=SVC(kernel="linear"),
      min_features_to_select= 100,
      cv = 5
  )
  select_features.fit(X_transform, y)

  filter = select_features.get_support(indices=True)
  return X.iloc[:, filter]

In [ ]:
# Embedded-based

def embedded_based(X, X_transform, y, quant):

  select_features = SelectFromModel(LogisticRegression(), max_features=quant)
  #select_features = SelectFromModel(SVC(kernel="linear", max_features=quant)
  #select_features = SelectFromModel(RandomForestClassifier(), max_features=quant)
  select_features.fit(X_transform, y)

  filter = select_features.get_support(indices=True)

  return X.iloc[:, filter]

**Métricas de Avaliação**


In [ ]:
def acuracia(resultados, classes):
    #resultados = cross_val_predict(modelo, textos, classes, cv=10)  
    return 'Acurácia do modelo: {} %'.format(round(metrics.accuracy_score(classes, resultados)*100, 2))

**Normalização:** é mudar os valores das colunas numéricas no conjunto de dados para usar uma escala comum, sem distorcer as diferenças nos intervalos de valores nem perder informações.

- **Normalização MinMax:** Para cada recurso, o valor mínimo desse recurso é transformado em 0, o valor máximo é transformado em 1 e todos os outros valores são transformados em um decimal entre 0 e 1.

In [ ]:
def minmax(data):
    scaler = MinMaxScaler()
    data_scaler = scaler.fit_transform(data)
    return data_scaler

## Criação dos modelos

**Serão feitas três análises:** Apenas com as variáveis socioeconômicas, com as variáveis de escolas e professores e depois com o dataset completo

Para cada uma serão criados modelos com todos os atributos e também com seleção de atributos, a fim de verificar o de maior acurácia

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [ ]:
def inicializar_modelos():
    naive = MultinomialNB()
    random = RandomForestClassifier(n_estimators=150)
    decision_tree = DecisionTreeClassifier()
    svm = SVC(kernel="linear")
    r_logistic = LogisticRegression(max_iter=250)
    knn = KNeighborsClassifier(n_neighbors=3)
    return naive, random, decision_tree, svm, r_logistic, knn

#### -Primeira análise: variáveis socioeconômicas derivadas da prova SAEB**

In [ ]:
#Foi retirada colunas do 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF'
columns = dataset.columns[3:234]

columns

Index(['ID_MUNICIPIO', 'ID_AREA', 'ID_DEPENDENCIA_ADM', 'ID_LOCALIZACAO',
       'IN_SITUACAO_CENSO', 'TX_RESP_Q001_A', 'TX_RESP_Q001_B',
       'TX_RESP_Q020_A', 'TX_RESP_Q020_B', 'TX_RESP_Q021_A',
       ...
       'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D',
       'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H',
       'TX_RESP_Q003_I', 'TX_RESP_Q003_J'],
      dtype='object', length=231)

In [ ]:
X =  dataset[columns]

In [ ]:
#Questões de sexo, raça, ano, mes de nascimento, conformidade com censo
question = ['TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D', 
            'TX_RESP_Q004_E', 'TX_RESP_Q004_F', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H', 'IN_SITUACAO_CENSO']

delete_at = ['IN_SITUACAO_CENSO']

X.drop(delete_at, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
filteredColumns = X.dtypes[X.dtypes == np.float]

listOfColumnNames = list(filteredColumns.index)

X[listOfColumnNames]

""
27000095
27001237
27002020
27002063
27002950
...
29840635
29848873
29886732
29903726


**Divisão entre conjunto de treino e teste**

In [ ]:
X_scaler = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 66.1 %
Naive Bayes: Acurácia do modelo: 62.71 %
Decision Tree: Acurácia do modelo: 57.63 %
SVM: Acurácia do modelo: 69.49 %
Logistic Regression: Acurácia do modelo: 67.8 %
KNN: Acurácia do modelo: 57.63 %


**-Primeira Análise: Aplicando Seleção de Atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X, y, 100)

X_selection

After selecting best 100 features: (587, 100)


,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,TX_RESP_Q001_A,TX_RESP_Q020_B,TX_RESP_Q021_B,TX_RESP_Q024_B,TX_RESP_Q027_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q018_B,TX_RESP_Q022_A,TX_RESP_Q022_B,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_A,TX_RESP_Q033_B,TX_RESP_Q035_C,TX_RESP_Q040_C,TX_RESP_Q041_C,TX_RESP_Q042_C,TX_RESP_Q048_B,TX_RESP_Q049_A,TX_RESP_Q050_B,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q056_A,TX_RESP_Q058_B,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q051_B,TX_RESP_Q051_C,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q059_B,...,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q013_E,TX_RESP_Q014_A,TX_RESP_Q014_B,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q014_E,TX_RESP_Q015_A,TX_RESP_Q015_B,TX_RESP_Q015_E,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q017_E,TX_RESP_Q036_A,TX_RESP_Q037_C,TX_RESP_Q037_D,TX_RESP_Q002_B,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_C,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_I
27000095,2700102,2,2,38,22,30,28,3,1,27,67,9,68,24,48,9,10,43,9,1,20,4,7,45,20,60,29,48,48,46,32,2,70,24,1,6,0,22,1,3,...,1,0,2,4,84,9,0,0,0,8,12,7,0,1,32,14,22,12,2,0,21,33,10,32,8,9,16,14,28,8,3,2,26,32,11,17,4,5,9,7
27001237,2701605,2,2,76,44,58,68,6,4,55,119,30,123,55,60,19,25,75,5,2,9,5,25,75,17,130,33,80,40,88,53,1,104,61,7,28,6,70,7,9,...,6,0,0,4,159,22,3,0,1,8,15,16,2,0,55,23,38,15,6,2,50,54,19,67,25,17,22,32,58,13,12,6,34,86,25,15,6,9,13,14
27002020,2705002,2,2,41,21,28,29,2,5,36,52,20,64,34,26,9,18,48,4,7,10,3,8,40,10,52,24,48,18,35,28,3,73,16,3,2,1,24,3,10,...,2,1,0,8,68,17,4,0,2,8,10,11,2,0,27,11,15,8,5,0,22,22,6,35,12,16,16,12,29,9,10,2,21,35,11,13,5,5,10,11
27002063,2705002,2,2,12,13,20,18,2,1,14,21,2,26,10,13,9,4,8,8,0,5,1,3,16,11,20,8,19,21,18,15,2,23,10,2,3,1,6,2,1,...,0,0,0,0,34,2,0,0,0,2,1,0,1,0,9,1,10,1,2,0,12,9,3,15,6,5,6,4,17,0,1,0,9,13,7,4,1,3,1,1
27002950,2706422,2,2,70,31,52,46,3,4,36,102,22,81,55,48,14,15,48,9,2,14,6,9,69,21,102,23,77,35,95,15,7,80,55,1,29,0,56,3,42,...,1,0,0,7,116,15,2,1,1,13,8,7,1,0,41,15,26,3,2,1,34,37,14,45,19,13,20,23,35,9,11,2,36,65,13,11,3,10,7,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,2923704,2,3,22,5,6,9,1,1,27,24,4,41,10,2,18,12,24,3,0,23,4,10,33,5,21,27,37,18,20,2,27,42,0,12,1,4,1,20,0,...,0,0,0,0,46,7,0,0,0,1,22,3,0,0,13,9,14,10,4,0,15,17,4,17,11,5,8,4,16,4,4,1,16,9,10,2,4,4,3,5
29848873,2923100,2,2,8,8,9,9,0,0,9,13,0,17,8,2,3,5,8,2,0,7,1,1,12,3,19,2,18,8,5,0,18,20,0,5,1,1,0,2,0,...,0,0,0,0,24,2,0,0,0,0,1,0,0,0,7,2,9,6,2,0,7,4,4,7,8,0,4,5,9,0,1,0,6,5,2,4,1,0,2,1
29886732,2902252,2,2,2,0,1,1,0,0,1,3,1,2,2,1,0,1,0,1,0,0,0,1,0,0,1,1,2,0,1,0,0,3,1,0,0,0,1,0,1,...,0,0,0,0,6,0,0,0,0,1,1,0,0,0,1,0,0,3,0,0,3,1,0,2,1,0,0,0,2,1,0,0,1,0,2,2,0,0,0,0
29903726,2933257,2,2,19,10,16,9,1,0,7,30,12,20,17,13,10,5,10,2,0,4,2,4,19,1,23,9,19,7,10,1,27,24,3,10,0,1,1,10,2,...,1,0,0,2,33,4,0,2,0,0,5,5,2,0,6,9,9,2,3,0,11,0,6,11,5,6,5,6,11,2,2,2,9,12,7,2,2,7,1,6


In [ ]:
X_scaler = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 66.1 %
Naive Bayes: Acurácia do modelo: 66.95 %
Decision Tree: Acurácia do modelo: 56.78 %
SVM: Acurácia do modelo: 71.19 %
Logistic Regression: Acurácia do modelo: 70.34 %
KNN: Acurácia do modelo: 65.25 %


**-Primeira Análise: Aplicando Seleção de Atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_scaler, y, 100)

X_embedded

,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_B,TX_RESP_Q029_A,TX_RESP_Q030_B,TX_RESP_Q031_B,TX_RESP_Q038_B,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q052_A,TX_RESP_Q018_B,TX_RESP_Q022_A,TX_RESP_Q032_A,TX_RESP_Q032_C,TX_RESP_Q040_B,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q042_A,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q048_B,TX_RESP_Q048_C,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,TX_RESP_Q054_A,TX_RESP_Q054_C,TX_RESP_Q056_C,TX_RESP_Q057_A,TX_RESP_Q039_A,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C
27000095,2700102,2,2,1,38,30,68,28,40,50,3,82,1,22,90,67,80,73,9,68,10,22,1,25,49,85,7,4,25,53,7,1,35,2,63,20,78,2,2,5,15,70,24,1
27001237,2701605,2,2,1,76,58,113,68,77,100,6,162,4,49,139,119,146,102,30,123,25,41,3,118,52,163,13,5,44,90,25,12,62,16,133,17,154,5,6,21,40,104,61,7
27002020,2705002,2,2,1,41,28,66,29,49,47,2,89,5,24,80,52,84,61,20,64,18,13,0,48,34,84,6,3,31,45,8,3,36,3,72,10,77,0,1,16,28,73,16,3
27002063,2705002,2,2,2,12,20,18,18,9,26,2,33,1,9,32,21,28,29,2,26,4,14,0,15,15,33,2,1,12,18,3,3,12,3,21,11,32,1,2,4,0,23,10,2
27002950,2706422,2,2,1,70,52,89,46,57,77,3,115,4,38,122,102,93,83,22,81,15,31,0,93,29,121,13,6,41,74,9,2,45,4,97,21,119,2,2,15,21,80,55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,2923704,2,3,2,22,6,45,9,39,14,1,51,1,8,48,24,48,31,4,41,12,11,1,18,13,47,3,4,19,28,10,2,16,2,45,5,49,0,0,11,6,42,0,12
29848873,2923100,2,2,2,8,9,15,9,9,15,0,22,0,8,24,13,21,15,0,17,5,3,0,15,2,23,1,1,8,13,1,0,9,0,20,3,22,0,0,3,2,20,0,5
29886732,2902252,2,2,2,2,1,4,1,3,2,0,4,0,0,4,3,5,1,1,2,1,1,0,4,0,5,0,0,1,1,1,0,2,0,3,0,1,0,0,0,4,3,1,0
29903726,2933257,2,2,1,19,16,30,9,16,22,1,34,0,14,24,30,26,24,12,20,5,5,1,27,8,36,1,2,11,15,4,4,9,4,29,1,25,2,3,4,1,24,3,10


In [ ]:
X_scaler_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_scaler_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 64.41 %
Naive Bayes: Acurácia do modelo: 59.32 %
Decision Tree: Acurácia do modelo: 49.15 %
SVM: Acurácia do modelo: 65.25 %
Logistic Regression: Acurácia do modelo: 65.25 %
KNN: Acurácia do modelo: 60.17 %


**-Primeira Análise: Aplicando Seleção de Atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_scaler, y, 100)

X_wrapper

,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_A,TX_RESP_Q026_B,...,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_B,TX_RESP_Q046_C,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q047_C,TX_RESP_Q048_A,TX_RESP_Q048_B,TX_RESP_Q048_C,TX_RESP_Q049_A,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,TX_RESP_Q050_C,TX_RESP_Q054_A,TX_RESP_Q054_B,TX_RESP_Q054_C,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q055_C,TX_RESP_Q056_A,TX_RESP_Q056_B,TX_RESP_Q056_C,TX_RESP_Q057_A,TX_RESP_Q057_B,TX_RESP_Q057_C,TX_RESP_Q058_A,TX_RESP_Q058_B,TX_RESP_Q058_C,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q045_D
27000095,2700102,2,2,1,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,24,4,38,48,...,7,4,17,61,3,25,53,3,84,7,1,45,35,2,63,20,1,78,3,2,60,29,2,48,30,2,5,71,4,32,48,2,15,46,32,2,70,24,1,1
27001237,2701605,2,2,1,76,108,144,44,130,58,113,68,77,100,177,6,167,16,162,21,179,4,133,49,43,139,55,119,146,36,102,75,153,30,4,123,55,5,103,60,...,13,5,25,111,19,44,90,17,121,25,12,75,62,16,133,17,14,154,5,5,130,33,4,80,66,6,21,121,8,112,40,6,40,88,53,1,104,61,7,9
27002020,2705002,2,2,1,41,60,78,21,70,28,66,29,49,47,95,2,88,9,89,8,91,5,73,24,13,80,36,52,84,8,61,24,79,20,1,64,34,1,62,26,...,6,3,36,43,3,31,45,5,73,8,3,40,36,3,72,10,2,77,3,0,52,24,0,48,28,1,16,55,1,55,18,2,28,35,28,3,73,16,3,2
27002063,2705002,2,2,2,12,24,22,13,15,20,18,18,9,26,34,2,32,4,33,3,35,1,26,9,2,32,14,21,28,7,29,5,33,2,0,26,10,0,13,13,...,2,1,6,23,3,12,18,2,29,3,3,16,12,3,21,11,1,32,1,1,20,8,2,19,10,2,4,26,1,10,21,1,0,18,15,2,23,10,2,1
27002950,2706422,2,2,1,70,72,109,31,88,52,89,46,57,77,134,3,123,13,115,22,133,4,98,38,14,122,36,102,93,46,83,52,116,22,3,81,55,3,76,48,...,13,6,26,91,4,41,74,4,122,9,2,69,45,4,97,21,4,119,5,2,102,23,2,77,38,2,15,97,4,79,35,5,21,95,15,7,80,55,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,2923704,2,3,2,22,31,50,5,49,6,45,9,39,14,54,1,51,3,51,4,53,1,46,8,5,48,27,24,48,7,31,20,48,4,3,41,10,4,35,2,...,3,4,12,33,2,19,28,1,37,10,2,33,16,2,45,5,1,49,3,0,21,27,3,37,13,0,11,38,1,34,18,1,6,20,2,27,42,0,12,1
29848873,2923100,2,2,2,8,18,18,8,17,9,15,9,9,15,25,0,24,0,22,3,25,0,17,8,1,24,9,13,21,3,15,9,25,0,0,17,8,0,20,2,...,1,1,8,13,0,8,13,0,22,1,0,12,9,0,20,3,0,22,0,0,19,2,0,18,4,0,3,18,0,14,8,0,2,5,0,18,20,0,5,0
29886732,2902252,2,2,2,2,4,5,0,4,1,4,1,3,2,5,0,5,0,4,0,4,0,5,0,0,4,1,3,5,0,1,2,5,1,0,2,2,1,3,1,...,0,0,0,2,0,1,1,0,2,1,0,0,2,0,3,0,0,1,1,0,1,1,0,2,1,0,0,2,0,2,0,0,4,1,0,0,3,1,0,1
29903726,2933257,2,2,1,19,23,32,10,26,16,30,9,16,22,38,1,33,6,34,5,39,0,25,14,13,24,7,30,26,13,24,13,28,12,1,20,17,2,16,13,...,1,2,6,20,4,11,15,4,24,4,4,19,9,4,29,1,3,25,3,2,23,9,3,19,9,3,4,22,3,22,7,3,1,10,1,27,24,3,10,1


In [ ]:
X_scaler_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_scaler_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 66.95 %
Naive Bayes: Acurácia do modelo: 61.86 %
Decision Tree: Acurácia do modelo: 50.0 %
SVM: Acurácia do modelo: 68.64 %
Logistic Regression: Acurácia do modelo: 67.8 %
KNN: Acurácia do modelo: 60.17 %


#### -Segunda análise: Análise com o dataset completo

In [ ]:
#Lembrar de tratar datas

'''question = ['TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q003_L', 'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D', 'CO_REGIAO',
            'TP_SITUACAO_FUNCIONAMENTO', 'TP_REGULAMENTACAO',
            'CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_DISTRITO', 'TP_LOCALIZACAO',  'CO_ORGAO_REGIONAL', 'TP_DEPENDENCIA',
            'TX_RESP_Q004_E', 'TX_RESP_Q004_F', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H', 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 
            'DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO', 'IN_SITUACAO_CENSO', 'NU_ANO_CENSO', 'CO_UF', 'CO_MUNICIPIO', 'Ideb-2017']'''

delete = ['CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_DISTRITO', 'TP_LOCALIZACAO',  'CO_ORGAO_REGIONAL', 'TP_DEPENDENCIA',
          'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 'IN_SITUACAO_CENSO', 'Ideb-2017']


#'NO_ENTIDADE', 

#X = dataset.drop(delete, axis=1)

X = dataset.drop(['Ideb-2017','IN_SITUACAO_CENSO'], axis=1)

In [ ]:
X

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,...,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,QtdeDocentesDivSocCulta,asQtdeDocentesEfetivos,QtdedocentesCLT
27000095,2017,2,27,2700102,2,2,1,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,24,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,172,0,62,0,0,103,0,172,0,0,148,0,172,508,172,0,24,0
27001237,2017,2,27,2701605,2,2,1,76,108,144,44,130,58,113,68,77,100,177,6,167,16,162,21,179,4,133,49,43,139,55,119,146,36,102,75,153,30,4,123,55,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,250,0,111,0,0,89,31,250,0,0,225,14,250,792,250,0,11,0
27002020,2017,2,27,2705002,2,2,1,41,60,78,21,70,28,66,29,49,47,95,2,88,9,89,8,91,5,73,24,13,80,36,52,84,8,61,24,79,20,1,64,34,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,222,0,100,0,0,86,0,222,0,0,217,0,222,652,222,0,5,0
27002063,2017,2,27,2705002,2,2,2,12,24,22,13,15,20,18,18,9,26,34,2,32,4,33,3,35,1,26,9,2,32,14,21,28,7,29,5,33,2,0,26,10,...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63,0,20,0,0,43,0,73,0,0,40,20,73,309,73,0,13,0
27002950,2017,2,27,2706422,2,2,1,70,72,109,31,88,52,89,46,57,77,134,3,123,13,115,22,133,4,98,38,14,122,36,102,93,46,83,52,116,22,3,81,55,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,117,0,72,7,0,36,0,121,0,0,84,0,121,442,121,0,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,2017,2,29,2923704,2,3,2,22,31,50,5,49,6,45,9,39,14,54,1,51,3,51,4,53,1,46,8,5,48,27,24,48,7,31,20,48,4,3,41,10,...,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,92,0,15,0,0,39,0,177,1,0,3,0,178,701,178,0,174,0
29848873,2017,2,29,2923100,2,2,2,8,18,18,8,17,9,15,9,9,15,25,0,24,0,22,3,25,0,17,8,1,24,9,13,21,3,15,9,25,0,0,17,8,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42,0,30,0,0,18,0,54,0,0,42,0,54,209,54,0,12,0
29886732,2017,2,29,2902252,2,2,2,2,4,5,0,4,1,4,1,3,2,5,0,5,0,4,0,4,0,5,0,0,4,1,3,5,0,1,2,5,1,0,2,2,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,93,0,56,8,0,127,0,199,0,0,193,0,199,389,199,0,6,0
29903726,2017,2,29,2933257,2,2,1,19,23,32,10,26,16,30,9,16,22,38,1,33,6,34,5,39,0,25,14,13,24,7,30,26,13,24,13,28,12,1,20,17,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42,0,38,0,0,19,0,63,0,0,47,0,63,194,63,0,16,0


In [ ]:
filteredColumns = X.dtypes[X.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

#X[listOfColumnNames].fillna(-1, inplace=True)

#X[listOfColumnNames] = X[listOfColumnNames].astype(int)

X[listOfColumnNames].isnull().sum()

Series([], dtype: float64)

In [ ]:
X_transform = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 63.56 %
Naive Bayes: Acurácia do modelo: 50.85 %
Decision Tree: Acurácia do modelo: 55.93 %
SVM: Acurácia do modelo: 67.8 %
Logistic Regression: Acurácia do modelo: 70.34 %
KNN: Acurácia do modelo: 61.86 %


**-Segunda Análise: Aplicando Seleção de Atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_transform, y, 100)

X_embedded

,ID_MUNICIPIO,TX_RESP_Q020_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q018_B,TX_RESP_Q022_A,TX_RESP_Q022_B,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_C,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q048_B,TX_RESP_Q049_B,TX_RESP_Q050_B,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q056_A,TX_RESP_Q058_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q051_C,TX_RESP_Q053_B,TX_RESP_Q060_A,TX_RESP_Q060_B,TX_RESP_Q060_D,TX_RESP_Q005_A,TX_RESP_Q006_A,TX_RESP_Q008_A,TX_RESP_Q008_C,...,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_C,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_J,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_TERMINO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,NU_SALAS_EXISTENTES,NU_EQUIP_TV,NU_EQUIP_IMPRESSORA,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FOTO,NU_COMPUTADOR,NU_COMP_ADMINISTRATIVO,NU_COMP_ALUNO,TP_INDIGENA_LINGUA,QtdeDocentesEsp,QtdeDocentesSemPos,QtdeDocentes,QtdeDocentesTempo,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,asQtdeDocentesEfetivos
27000095,2700102,22,68,28,27,67,80,9,68,24,48,9,22,25,49,20,7,4,17,7,35,20,60,29,48,32,46,32,2,24,1,0,22,11,45,0,6,31,9,1,...,33,10,32,17,9,16,14,28,3,2,26,32,17,4,5,7,11,1513296000000000000,2701,27001,27,2700102,270010205,10,2,2,2,0,28,4,24,-1,62,103,172,148,172,508,172,24
27001237,2701605,44,113,68,55,119,146,30,123,55,60,19,41,118,52,9,13,5,25,25,62,17,130,33,80,112,88,53,1,61,7,6,70,27,64,13,7,66,4,13,...,54,19,67,28,17,22,32,58,12,6,34,86,15,6,9,11,11,1513296000000000000,2701,27001,27,2701605,270160505,6,6,2,6,1,11,3,0,-1,111,89,250,225,250,792,250,11
27002020,2705002,21,66,29,36,52,84,20,64,34,26,9,13,48,34,10,6,3,36,8,36,10,52,24,48,55,35,28,3,16,3,1,24,20,40,2,6,33,5,9,...,22,6,35,9,16,16,12,29,10,2,21,35,13,5,5,9,11,1513296000000000000,2701,27001,27,2705002,270500205,7,3,3,2,0,23,5,18,-1,100,86,222,217,222,652,222,5
27002063,2705002,13,18,18,14,21,28,2,26,10,13,9,14,15,15,5,2,1,6,3,12,11,20,8,19,10,18,15,2,10,2,1,6,7,20,2,2,12,0,0,...,9,3,15,0,5,6,4,17,1,0,9,13,4,1,3,5,11,1513296000000000000,2701,27001,27,2705002,270500205,7,1,1,0,0,4,4,0,-1,20,43,73,40,73,309,73,13
27002950,2706422,31,89,46,36,102,93,22,81,55,48,14,31,93,29,14,13,6,26,9,45,21,102,23,77,79,95,15,7,55,1,0,56,11,52,10,11,56,8,4,...,37,14,45,23,13,20,23,35,11,2,36,65,11,3,10,12,11,1513296000000000000,2701,27001,27,2706422,270642205,6,2,1,4,1,23,4,19,-1,72,36,121,84,121,442,121,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,2923704,5,45,9,27,24,48,4,41,10,2,18,11,18,13,23,3,4,12,10,16,5,21,27,37,34,20,2,27,0,12,4,1,20,5,2,6,20,2,3,...,17,4,17,4,5,8,4,16,4,1,16,9,2,4,4,4,2,1513900800000000000,2902,29007,29,2923704,292370410,13,1,0,1,1,11,1,10,0,15,39,177,3,178,701,178,174
29848873,2923100,8,15,9,9,13,21,0,17,8,2,3,3,15,2,7,1,1,8,1,9,3,19,2,18,14,5,0,18,0,5,1,0,9,5,0,1,11,0,0,...,4,4,7,2,0,4,5,9,1,0,6,5,4,1,0,5,18,1513123200000000000,2904,29015,29,2923100,292310005,7,12,6,1,1,11,3,8,0,30,18,54,42,54,209,54,12
29886732,2902252,0,4,1,1,3,5,1,2,2,1,0,1,4,0,0,0,0,0,1,2,0,1,1,2,2,1,0,0,1,0,0,1,2,0,1,0,3,0,0,...,1,0,2,0,0,0,0,2,0,0,1,0,2,0,0,0,5,1513123200000000000,2907,29031,29,2902252,290225205,6,7,3,0,0,25,4,21,0,56,127,199,193,199,389,199,6
29903726,2933257,10,30,9,7,30,26,12,20,17,13,10,5,27,8,4,1,2,6,4,9,1,23,9,19,22,10,1,27,3,10,1,1,7,7,11,1,20,1,4,...,0,6,11,7,6,5,6,11,2,2,9,12,2,2,7,1,7,1513123200000000000,2907,29032,29,2933257,293325705,6,10,2,0,0,12,3,9,0,38,19,63,47,63,194,63,16


In [ ]:
X_transform_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_transform_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 67.8 %
Naive Bayes: Acurácia do modelo: 62.71 %
Decision Tree: Acurácia do modelo: 55.93 %
SVM: Acurácia do modelo: 68.64 %
Logistic Regression: Acurácia do modelo: 71.19 %
KNN: Acurácia do modelo: 64.41 %


**-Segunda Análise: Aplicando Seleção de Atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_transform, y, 100)

X_wrapper

,ID_UF,ID_AREA,ID_DEPENDENCIA_ADM,TX_RESP_Q001_A,TX_RESP_Q020_B,TX_RESP_Q029_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q018_B,TX_RESP_Q022_B,TX_RESP_Q026_A,TX_RESP_Q026_C,TX_RESP_Q032_B,TX_RESP_Q033_B,TX_RESP_Q033_C,TX_RESP_Q041_C,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_B,TX_RESP_Q047_B,TX_RESP_Q048_B,TX_RESP_Q049_A,TX_RESP_Q049_C,TX_RESP_Q054_B,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q056_A,TX_RESP_Q056_B,TX_RESP_Q057_B,TX_RESP_Q058_B,TX_RESP_Q039_C,TX_RESP_Q051_B,TX_RESP_Q051_C,TX_RESP_Q059_B,TX_RESP_Q060_A,TX_RESP_Q005_A,TX_RESP_Q007_D,TX_RESP_Q007_E,TX_RESP_Q008_C,TX_RESP_Q008_E,...,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_G,TX_RESP_Q003_L,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_TERMINO,CO_UF,TP_DEPENDENCIA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_AGUA_REDE_PUBLICA,IN_AGUA_INEXISTENTE,IN_LIXO_ENTERRA,IN_LIXO_OUTROS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_PARQUE_INFANTIL,IN_BANHEIRO_EI,IN_ALOJAM_ALUNO,IN_ALOJAM_PROFESSOR,IN_LAVANDERIA,NU_EQUIP_DVD,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_SOM,TP_AEE,IN_COMUM_PRE,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_NORMAL,IN_COMUM_PROF,QtdeDocentesMest,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeAlunosTotal,QtdeDocentesDivSocCulta,QtdedocentesCLT
27000095,27,2,2,38,22,13,1,27,9,24,38,9,55,43,30,20,7,4,61,53,7,45,2,3,60,29,48,30,71,48,32,6,0,3,11,6,0,0,1,1,...,4,5,7,9,9,7,11,1513296000000000000,27,2,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,148,-1,508,0,-1
27001237,27,2,2,76,44,21,4,55,30,55,103,19,98,75,63,9,13,5,111,90,25,75,16,5,130,33,80,66,121,40,53,28,6,9,27,7,2,0,13,0,...,6,9,14,24,19,18,11,1513296000000000000,27,2,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,6,4,0,0,0,0,1,0,0,0,225,14,792,0,-1
27002020,27,2,2,41,21,8,5,36,20,34,62,9,60,48,25,10,6,3,43,45,8,40,3,3,52,24,48,28,55,18,28,2,1,10,20,6,0,0,9,0,...,5,5,8,15,9,5,11,1513296000000000000,27,2,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,1,0,0,0,217,-1,652,0,-1
27002063,27,2,2,12,13,3,1,14,2,10,13,9,15,8,21,5,2,1,23,18,3,16,3,1,20,8,19,10,26,21,15,3,1,1,7,2,0,1,0,1,...,1,3,2,6,5,1,11,1513296000000000000,27,2,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,40,20,309,0,-1
27002950,27,2,2,70,31,22,4,36,22,55,76,14,82,48,48,14,13,6,91,74,9,69,4,5,102,23,77,38,97,35,15,29,0,42,11,11,1,0,4,0,...,3,10,11,19,8,7,11,1513296000000000000,27,2,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,1,2,0,0,0,1,0,0,7,84,-1,442,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,29,2,3,22,5,4,1,27,4,10,35,18,28,24,15,23,3,4,33,28,10,33,2,3,21,27,37,13,38,18,2,1,4,0,20,6,0,0,3,0,...,4,4,6,1,5,3,2,1513900800000000000,29,3,1,1,0,0,0,1,0,0,0,0,1,1,0,0,1,1,2,1,0,1,1,1,0,0,0,3,0,701,0,0
29848873,29,2,2,8,8,3,0,9,0,8,20,3,15,8,11,7,1,1,13,13,1,12,0,0,19,2,18,4,18,8,0,1,1,0,9,1,0,0,0,0,...,1,0,1,3,3,3,18,1513123200000000000,29,2,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,42,0,209,0,0
29886732,29,2,2,2,0,0,0,1,1,2,3,0,0,0,1,0,0,0,2,1,1,0,0,1,1,1,2,1,2,0,0,0,0,1,2,0,0,0,0,0,...,0,0,1,1,0,1,5,1513123200000000000,29,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,193,0,389,0,0
29903726,29,2,2,19,10,5,0,7,12,17,16,10,23,10,18,4,1,2,20,15,4,19,4,3,23,9,19,9,22,7,1,0,1,2,7,1,0,0,4,0,...,2,7,4,3,2,3,7,1513123200000000000,29,2,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,47,0,194,0,0


In [ ]:
X_transform_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_transform_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 69.49 %
Naive Bayes: Acurácia do modelo: 64.41 %
Decision Tree: Acurácia do modelo: 61.02 %
SVM: Acurácia do modelo: 77.12 %
Logistic Regression: Acurácia do modelo: 77.97 %
KNN: Acurácia do modelo: 65.25 %


**-Segunda Análise: Aplicando Seleção de Atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X, y, 100)

X_selection

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   1 234 236 239 247 248 249 250 251 252 253 254 260 261 262 263 264
 265 278 280 303 369 370 372 373 383 384 385 386 387 388 389 393 394 395
 397 399 413] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,TX_RESP_Q020_B,TX_RESP_Q024_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q018_B,TX_RESP_Q026_C,TX_RESP_Q035_C,TX_RESP_Q040_C,TX_RESP_Q041_C,TX_RESP_Q042_C,TX_RESP_Q050_B,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q058_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q051_C,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q060_D,TX_RESP_Q005_A,TX_RESP_Q005_D,TX_RESP_Q008_A,TX_RESP_Q009_A,TX_RESP_Q012_A,TX_RESP_Q013_A,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q013_E,TX_RESP_Q014_A,TX_RESP_Q014_B,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q014_E,...,TX_RESP_Q004_G,TX_RESP_Q004_H,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_COBERTA,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,IN_BANHEIRO_CHUVEIRO,IN_REFEITORIO,IN_DESPENSA,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_AREA_VERDE,IN_LAVANDERIA,IN_EQUIP_TV,IN_EQUIP_MULTIMIDIA,IN_EQUIP_FOTO,NU_EQUIP_DVD,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_SOM,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FOTO,NU_COMPUTADOR,NU_COMP_ADMINISTRATIVO,NU_COMP_ALUNO,NU_FUNCIONARIOS,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_PROF
27000095,27,2700102,2,2,22,28,1,27,9,9,9,1,20,4,20,60,29,48,32,2,24,1,0,22,1,0,6,2,9,64,85,80,1,0,2,4,84,9,0,0,...,4,5,11,1494374400000000000,1513296000000000000,2701,27001,27,2700102,270010205,2,1,1,0,0,1,1,1,0,1,1,1,0,0,0,1,1,0,1,0,1,2,0,28,4,24,50,0,1,0
27001237,27,2701605,2,2,44,68,4,55,30,19,5,2,9,5,17,130,33,40,53,1,61,7,6,70,7,13,7,4,4,115,143,145,6,0,0,4,159,22,3,0,...,6,9,11,1486339200000000000,1513296000000000000,2701,27001,27,2701605,270160505,2,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1,6,4,0,6,1,11,3,0,38,0,1,0
27002020,27,2705002,2,2,21,29,5,36,20,9,4,7,10,3,10,52,24,18,28,3,16,3,1,24,3,2,6,2,5,61,72,78,2,1,0,8,68,17,4,0,...,5,5,11,1488758400000000000,1513296000000000000,2701,27001,27,2705002,270500205,2,1,0,0,0,1,1,0,0,1,0,1,0,0,0,1,1,0,1,0,2,2,0,23,5,18,24,0,1,0
27002063,27,2705002,2,2,13,18,1,14,2,9,8,0,5,1,11,20,8,21,15,2,10,2,1,6,2,2,2,1,0,27,25,30,0,0,0,0,34,2,0,0,...,1,3,11,1488758400000000000,1513296000000000000,2701,27001,27,2705002,270500205,2,1,0,0,0,1,1,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,4,4,0,28,1,1,0
27002950,27,2706422,2,2,31,46,4,36,22,14,9,2,14,6,21,102,23,35,15,7,55,1,0,56,3,10,11,2,8,101,120,112,1,0,0,7,116,15,2,1,...,3,10,11,1491782400000000000,1513296000000000000,2701,27001,27,2706422,270642205,2,1,0,0,0,1,0,1,0,1,0,1,1,0,0,1,1,1,2,1,2,4,1,23,4,19,38,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,29,2923704,2,3,5,9,1,27,4,18,3,0,23,4,5,21,27,18,2,27,0,12,4,1,20,2,6,0,2,35,39,43,0,0,0,0,46,7,0,0,...,4,4,2,1489363200000000000,1513900800000000000,2902,29007,29,2923704,292370410,3,1,0,0,1,0,0,1,1,1,1,1,0,0,1,1,1,1,1,2,1,1,1,11,1,10,78,1,1,0
29848873,29,2923100,2,2,8,9,0,9,0,3,2,0,7,1,3,19,2,8,0,18,0,5,1,0,2,0,1,1,0,18,20,22,0,0,0,0,24,2,0,0,...,1,0,18,1486339200000000000,1513123200000000000,2904,29015,29,2923100,292310005,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,1,1,11,3,8,24,0,1,0
29886732,29,2902252,2,2,0,1,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,3,1,2,0,0,0,0,6,0,0,0,...,0,0,5,1486339200000000000,1513123200000000000,2907,29031,29,2902252,290225205,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,25,4,21,47,0,0,0
29903726,29,2933257,2,2,10,9,0,7,12,10,2,0,4,2,1,23,9,7,1,27,3,10,1,1,10,11,1,0,1,21,35,29,1,0,0,2,33,4,0,2,...,2,7,7,1486339200000000000,1513123200000000000,2907,29032,29,2933257,293325705,2,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,12,3,9,24,0,1,0


In [ ]:
X_transform = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 93.19 %
Naive Bayes: Acurácia do modelo: 64.05 %
Decision Tree: Acurácia do modelo: 90.29 %
SVM: Acurácia do modelo: 72.91 %
Logistic Regression: Acurácia do modelo: 73.42 %
KNN: Acurácia do modelo: 72.57 %


**Segunda Análise: Unindo subconjunto de melhores atributos**

In [ ]:
X_all = pd.concat([X_selection, X_wrapper], axis=1)

X_all = pd.concat([X_all, X_embedded], axis=1)

In [ ]:
#Removendo colunas duplicadas
X_new_all = X_all.T.drop_duplicates().T

In [ ]:
X_transform = minmax(X_new_all)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 92.67 %
Naive Bayes: Acurácia do modelo: 63.54 %
Decision Tree: Acurácia do modelo: 90.12 %
SVM: Acurácia do modelo: 80.75 %
Logistic Regression: Acurácia do modelo: 80.07 %
KNN: Acurácia do modelo: 72.74 %


Resultado com Todos atributos

- Random forest: Acurácia do modelo: 93.36 %
- Naive Bayes: Acurácia do modelo: 61.67 %
- Decision Tree: Acurácia do modelo: 91.14 %
- SVM: Acurácia do modelo: 79.56 %
- Logistic Regression: Acurácia do modelo: 79.9 %
- KNN: Acurácia do modelo: 74.45 %

**Segunda Análise: Aplicando a técnica merge**

Link da Dissertação: https://www.dcc.ufmg.br/pos/cursos/defesas/1930M.PDF


In [ ]:
#Metodo Merge combina os atributos mais frequentes nos melhores conjuntos de seleção 
#Cada atributo irá apresentar um score de acordo com sua presença em um subconjunto de dados
def metodo_merge(selection, embedded, wrapper, ranking):
  for atributo in selection:
    score = ranking[atributo]
    if (atributo in embedded):
      score += 1
      embedded.remove(atributo)
    if (atributo in wrapper):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score

  for atributo in embedded:
    score = ranking[atributo]
    if (atributo in selection):
      score += 1
      embedded.remove(atributo)
    if (atributo in wrapper):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score

  for atributo in wrapper:
    score = ranking[atributo]
    if (atributo in selection):
      score += 1
      embedded.remove(atributo)
    if (atributo in embedded):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score
 
  return ranking      

In [ ]:
#Gerando Listas com os melhores atributos de cada abordagem
selection = list(X_selection.columns)
embedded = list(X_embedded.columns)
wrapper = list(X_wrapper.columns)

#Criando dicionário
values = [1]*300

all = selection + embedded + wrapper

ranking = dict(zip(all, values))

ranking = metodo_merge(selection, embedded, wrapper, ranking)

#Filtrando os atributos que estão em mais de um subconjunto

dic_aux = ranking.items()

filtered = {key:value for key, value in dic_aux if value > 1}

#Gerando X após o merge

X_merge = dataset[list(filtered.keys())]

In [ ]:
X_transform = minmax(X_merge)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 94.38 %
Naive Bayes: Acurácia do modelo: 66.95 %
Decision Tree: Acurácia do modelo: 91.14 %
SVM: Acurácia do modelo: 76.66 %
Logistic Regression: Acurácia do modelo: 78.36 %
KNN: Acurácia do modelo: 77.0 %


In [ ]:
dados_para_exportar = pd.DataFrame(columns=X_merge.columns, data=X_merge.values)

dados_para_exportar

,ID_UF,ID_DEPENDENCIA_ADM,TX_RESP_Q020_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q018_B,TX_RESP_Q040_C,TX_RESP_Q042_C,TX_RESP_Q055_B,TX_RESP_Q058_B,TX_RESP_Q039_C,TX_RESP_Q051_C,TX_RESP_Q060_D,TX_RESP_Q005_A,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q015_B,TX_RESP_Q015_E,TX_RESP_Q017_C,TX_RESP_Q017_E,TX_RESP_Q036_A,TX_RESP_Q002_B,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_G,TX_RESP_Q004_G,TX_RESP_Q004_H,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_TERMINO,CO_UF,TP_DEPENDENCIA,TP_OCUPACAO_PREDIO_ESCOLAR,IN_SALA_ATENDIMENTO_ESPECIAL,NU_EQUIP_DVD,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_SOM,...,TX_RESP_Q026_A,TX_RESP_Q026_C,TX_RESP_Q033_B,TX_RESP_Q042_B,TX_RESP_Q046_B,TX_RESP_Q047_B,TX_RESP_Q049_A,TX_RESP_Q055_A,TX_RESP_Q056_A,TX_RESP_Q057_B,TX_RESP_Q051_B,TX_RESP_Q060_A,TX_RESP_Q007_D,TX_RESP_Q007_E,TX_RESP_Q008_C,TX_RESP_Q008_E,TX_RESP_Q009_C,TX_RESP_Q011_D,TX_RESP_Q012_B,TX_RESP_Q036_E,TX_RESP_Q002_A,TX_RESP_Q004_C,TX_RESP_Q003_E,TX_RESP_Q003_G,TP_REGULAMENTACAO,IN_LOCAL_FUNC_GALPAO,IN_LOCAL_FUNC_OUTROS,IN_AGUA_REDE_PUBLICA,IN_AGUA_INEXISTENTE,IN_LIXO_OUTROS,IN_BANHEIRO_EI,IN_ALOJAM_PROFESSOR,IN_LAVANDERIA,TP_AEE,IN_COMUM_PRE,IN_COMUM_MEDIO_NORMAL,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeAlunosTotal,QtdeDocentesDivSocCulta
0,27,2,22,1,27,9,1,4,29,48,32,0,0,6,1,0,9,0,8,12,0,1,32,12,2,0,9,16,26,4,5,11,1513296000000000000,27,2,1,0,1,0,1,...,38,9,43,7,61,53,45,60,48,71,6,11,0,0,1,1,2,0,11,15,7,32,9,9,1,0,0,1,0,0,0,0,0,0,0,0,148,-1,508,0
1,27,2,44,4,55,30,2,5,33,40,53,6,13,7,6,0,22,3,8,15,2,0,55,15,6,2,17,22,34,6,9,11,1513296000000000000,27,2,1,0,6,4,0,...,103,19,75,13,111,90,75,130,80,121,28,27,2,0,13,0,1,0,36,12,29,86,24,19,1,0,0,1,0,0,0,0,0,0,0,0,225,14,792,0
2,27,2,21,5,36,20,7,3,24,18,28,1,2,6,2,1,17,4,8,10,2,0,27,8,5,0,16,16,21,5,5,11,1513296000000000000,27,2,1,0,1,0,2,...,62,9,48,6,43,45,40,52,48,55,2,20,0,0,9,0,3,0,28,16,15,35,15,9,2,0,0,1,0,0,0,0,0,0,0,0,217,-1,652,0
3,27,2,13,1,14,2,0,1,8,21,15,1,2,2,0,0,2,0,2,1,1,0,9,1,2,0,5,6,9,1,3,11,1513296000000000000,27,2,1,0,1,0,1,...,13,9,8,2,23,18,16,20,19,26,3,7,0,1,0,1,0,1,11,5,5,13,6,5,1,0,0,1,0,0,0,0,0,0,0,0,40,20,309,0
4,27,2,31,4,36,22,2,6,23,35,15,0,10,11,1,0,15,2,13,8,1,0,41,3,2,1,13,20,36,3,10,11,1513296000000000000,27,2,1,0,2,1,2,...,76,14,48,13,91,74,69,102,77,97,29,11,1,0,4,0,1,0,20,25,12,65,19,8,2,0,0,1,0,0,0,0,0,0,0,0,84,-1,442,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,29,3,5,1,27,4,0,4,27,18,2,4,2,6,0,0,7,0,1,22,0,0,13,10,4,0,5,8,16,4,4,2,1513900800000000000,29,3,1,0,1,2,1,...,35,18,24,3,33,28,33,21,37,38,1,20,0,0,3,0,2,0,15,6,4,9,1,5,1,0,0,1,0,0,1,0,1,0,1,0,3,0,701,0
583,29,2,8,0,9,0,0,1,2,8,0,1,0,1,0,0,2,0,0,1,0,0,7,6,2,0,0,4,6,1,0,18,1513123200000000000,29,2,1,0,1,0,1,...,20,3,8,1,13,13,12,19,18,18,1,9,0,0,0,0,0,0,6,2,4,5,3,3,2,0,0,1,0,0,0,0,0,0,0,0,42,0,209,0
584,29,2,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,3,0,0,0,0,1,0,0,5,1513123200000000000,29,2,1,0,0,0,0,...,3,0,0,0,2,1,0,1,2,2,0,2,0,0,0,0,0,0,5,0,2,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,193,0,389,0
585,29,2,10,0,7,12,0,2,9,7,1,1,11,1,1,0,4,0,0,5,2,0,6,2,3,0,6,5,9,2,7,7,1513123200000000000,29,2,1,1,1,0,0,...,16,10,10,1,20,15,19,23,19,22,0,7,0,0,4,0,3,0,5,8,12,12,3,2,1,0,0,1,0,0,0,0,0,0,0,0,47,0,194,0


In [ ]:
#dados_para_exportar.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/resultado_analise_merge_dataset_completo.csv')
dados_para_exportar.to_csv('/content/drive/MyDrive/TCC/resultado_analise_merge_dataset_completo.csv')

#### -Terceira análise: variáveis das escolas e professores

In [ ]:
columns_escolas = dataset.columns[237:404]

X_escolas = dataset[columns_escolas]

X_escolas

,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_POCO_ARTESIANO,IN_AGUA_CACIMBA,IN_AGUA_FONTE_RIO,...,IN_EDUCACAO_INDIGENA,TP_INDIGENA_LINGUA,CO_LINGUA_INDIGENA,IN_BRASIL_ALFABETIZADO,IN_FINAL_SEMANA,IN_FORMACAO_ALTERNANCIA,IN_MEDIACAO_PRESENCIAL,IN_MEDIACAO_SEMIPRESENCIAL,IN_MEDIACAO_EAD,IN_ESPECIAL_EXCLUSIVA,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout
27000095,1,1494374400000000000,1513296000000000000,2,2701,27001,27,2700102,270010205,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,...,0,-1,-1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,172,0,62,0,0
27001237,1,1486339200000000000,1513296000000000000,2,2701,27001,27,2701605,270160505,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,0,0,0,0,0,0,0,-1,1,0,0,1,1,0,1,0,...,0,-1,-1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,250,0,111,0,0
27002020,1,1488758400000000000,1513296000000000000,2,2701,27001,27,2705002,270500205,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,1,1,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,...,0,-1,-1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,222,0,100,0,0
27002063,1,1488758400000000000,1513296000000000000,2,2701,27001,27,2705002,270500205,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,...,0,-1,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63,0,20,0,0
27002950,1,1491782400000000000,1513296000000000000,2,2701,27001,27,2706422,270642205,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,1,1,0,0,0,0,0,0,0,-1,0,0,1,1,1,0,0,0,...,0,-1,-1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,117,0,72,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,1,1489363200000000000,1513900800000000000,2,2902,29007,29,2923704,292370410,3,2,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,1,0,1,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,92,0,15,0,0
29848873,1,1486339200000000000,1513123200000000000,2,2904,29015,29,2923100,292310005,2,2,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42,0,30,0,0
29886732,1,1486339200000000000,1513123200000000000,2,2907,29031,29,2902252,290225205,2,2,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,93,0,56,8,0
29903726,1,1486

In [ ]:
X_escolas.drop(columns=['DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO'], axis=1, inplace=True)

X_transform = minmax(X_escolas)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 63.56 %
Naive Bayes: Acurácia do modelo: 61.86 %
Decision Tree: Acurácia do modelo: 54.24 %
SVM: Acurácia do modelo: 64.41 %
Logistic Regression: Acurácia do modelo: 66.95 %
KNN: Acurácia do modelo: 62.71 %


**Aplicando seleção de atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X_escolas, y, 100)

After selecting best 100 features: (587, 100)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   1  22  23  24  25  26  27  40  42  65 131 132 134 135 145 146 147
 148 149 150 151 155 156 157 159 161] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 62.71 %
Naive Bayes: Acurácia do modelo: 61.86 %
Decision Tree: Acurácia do modelo: 52.54 %
SVM: Acurácia do modelo: 64.41 %
Logistic Regression: Acurácia do modelo: 66.95 %
KNN: Acurácia do modelo: 62.71 %


**Aplicando seleção de atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_transform, y, 100)

X_embedded

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q021_A,TX_RESP_Q030_A,TX_RESP_Q031_A,TX_RESP_Q044_B,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_B,TX_RESP_Q033_A,TX_RESP_Q033_B,TX_RESP_Q034_A,TX_RESP_Q034_B,TX_RESP_Q035_B,TX_RESP_Q035_C,TX_RESP_Q040_A,TX_RESP_Q040_B,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_A,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_B,TX_RESP_Q047_C,TX_RESP_Q048_A,TX_RESP_Q048_B,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_B,TX_RESP_Q054_A,TX_RESP_Q055_B,TX_RESP_Q056_A,TX_RESP_Q057_B,TX_RESP_Q058_A,TX_RESP_Q058_C,TX_RESP_Q045_D,TX_RESP_Q051_B,TX_RESP_Q051_C,TX_RESP_Q053_A,TX_RESP_Q053_D,TX_RESP_Q059_A,TX_RESP_Q059_D,TX_RESP_Q060_B,TX_RESP_Q005_A,TX_RESP_Q005_B,TX_RESP_Q005_D,TX_RESP_Q006_B,TX_RESP_Q006_D,TX_RESP_Q007_A,TX_RESP_Q007_B,TX_RESP_Q007_C,TX_RESP_Q007_D,TX_RESP_Q009_B,TX_RESP_Q009_E,TX_RESP_Q010_B,TX_RESP_Q010_C,TX_RESP_Q010_D,TX_RESP_Q010_E,TX_RESP_Q012_C,TX_RESP_Q013_D,TX_RESP_Q014_A,TX_RESP_Q014_C
27000095,1,38,59,68,93,73,16,20,86,2,68,24,4,48,9,43,36,46,27,9,94,1,49,20,85,4,17,61,3,84,7,35,2,20,78,29,48,71,32,2,1,6,0,71,2,89,2,45,6,82,2,63,0,27,68,3,0,32,0,9,0,0,0,0,0,4,9
27001237,1,76,108,130,179,133,36,75,153,4,123,55,5,60,25,75,64,91,58,5,178,3,52,9,163,5,25,111,17,121,25,62,16,17,154,33,80,121,112,6,9,28,6,98,4,164,6,64,7,139,4,106,0,60,121,5,2,65,1,11,4,0,0,3,0,4,22
27002020,1,41,60,70,91,73,8,24,79,1,64,34,1,26,16,48,38,46,28,4,87,0,34,10,84,3,36,43,5,73,8,36,3,10,77,24,48,55,55,2,2,2,1,58,1,77,1,40,6,72,2,57,1,46,44,11,0,37,0,15,1,0,0,0,1,8,17
27002063,2,12,24,15,35,26,7,5,33,0,26,10,0,13,3,8,12,18,12,8,36,0,15,5,33,1,6,23,2,29,3,12,3,11,32,8,19,26,10,1,1,3,1,26,2,33,2,20,2,31,1,22,0,15,19,1,0,9,0,2,0,0,0,0,0,0,2
27002950,1,70,72,88,133,98,46,52,116,3,81,55,3,48,27,48,45,66,45,9,138,0,29,14,121,6,26,91,4,122,9,45,4,21,119,23,77,97,79,5,4,29,0,72,5,85,3,52,11,111,2,80,0,55,83,4,1,40,0,9,0,0,0,1,0,7,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,2,22,31,49,53,46,7,20,48,3,41,10,4,2,9,24,22,28,14,3,54,1,13,23,47,4,12,33,1,37,10,16,2,5,49,27,37,38,34,1,1,1,4,33,0,45,0,5,6,41,0,31,0,24,29,1,0,17,0,16,1,0,0,1,0,0,7
29848873,2,8,18,17,25,17,3,9,25,0,17,8,0,2,3,8,11,11,6,2,25,0,2,7,23,1,8,13,0,22,1,9,0,3,22,2,18,18,14,0,0,1,1,22,0,24,0,5,1,22,1,13,1,9,16,0,0,8,0,1,1,0,0,0,0,0,2
29886732,2,2,4,4,4,5,0,2,5,0,2,2,1,1,1,0,2,3,0,1,5,0,0,0,5,0,0,2,0,2,1,2,0,0,1,1,2,2,2,0,1,0,0,2,1,2,1,0,0,4,0,3,0,2,4,0,0,2,0,2,0,0,0,0,0,0,0
29903726,1,19,23,26,39,25,13,13,28,1,20,17,2,13,4,10,18,10,9,2,38,1,8,4,36,2,6,20,4,24,4,9,4,1,25,9,19,22,22,3,1,0,1,24,3,29,3,7,1,33,0,19,0,16,26,0,0,14,0,9,1,0,1,2,0,2,4


In [ ]:
X_transform_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_transform_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 66.1 %
Naive Bayes: Acurácia do modelo: 68.64 %
Decision Tree: Acurácia do modelo: 58.47 %
SVM: Acurácia do modelo: 70.34 %
Logistic Regression: Acurácia do modelo: 72.88 %
KNN: Acurácia do modelo: 61.02 %


**Aplicando seleção de atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_transform, y, 100)

X_wrapper

,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q025_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q031_A,TX_RESP_Q018_A,TX_RESP_Q018_C,TX_RESP_Q022_C,TX_RESP_Q033_A,TX_RESP_Q033_B,TX_RESP_Q035_C,TX_RESP_Q040_A,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_A,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_B,TX_RESP_Q046_C,TX_RESP_Q047_C,TX_RESP_Q048_A,TX_RESP_Q048_B,TX_RESP_Q048_C,TX_RESP_Q049_A,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,...,TX_RESP_Q045_C,TX_RESP_Q051_A,TX_RESP_Q051_B,TX_RESP_Q051_C,TX_RESP_Q051_D,TX_RESP_Q053_A,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q053_D,TX_RESP_Q059_A,TX_RESP_Q059_B,TX_RESP_Q059_C,TX_RESP_Q059_D,TX_RESP_Q060_A,TX_RESP_Q060_B,TX_RESP_Q060_C,TX_RESP_Q060_D,TX_RESP_Q005_A,TX_RESP_Q005_B,TX_RESP_Q005_D,TX_RESP_Q005_E,TX_RESP_Q006_A,TX_RESP_Q006_E,TX_RESP_Q007_B,TX_RESP_Q007_C,TX_RESP_Q007_E,TX_RESP_Q008_A,TX_RESP_Q009_B,TX_RESP_Q009_C,TX_RESP_Q010_A,TX_RESP_Q010_B,TX_RESP_Q010_C,TX_RESP_Q010_D,TX_RESP_Q012_C,TX_RESP_Q012_D,TX_RESP_Q012_E,TX_RESP_Q014_A,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q014_E
27000095,1,38,59,75,22,30,68,40,3,87,13,93,73,86,2,4,9,43,9,94,1,1,25,49,20,85,7,4,17,61,3,3,84,7,1,45,35,2,63,20,...,1,88,6,0,2,71,22,1,2,89,3,0,2,11,45,39,0,6,82,2,0,31,1,68,3,0,9,32,2,89,9,0,0,0,1,0,4,9,0,0
27001237,1,76,108,144,44,58,113,77,6,167,21,179,133,153,4,5,25,75,5,178,3,2,118,52,9,163,13,5,25,111,19,17,121,25,12,75,62,16,133,17,...,7,137,28,6,11,98,70,7,4,164,9,3,6,27,64,76,13,7,139,4,0,66,0,121,5,0,4,65,1,171,11,4,0,3,2,1,4,22,3,0
27002020,1,41,60,78,21,28,66,49,2,88,8,91,73,79,1,1,16,48,4,87,0,7,48,34,10,84,6,3,36,43,3,5,73,8,3,40,36,3,72,10,...,3,85,2,1,2,58,24,3,1,77,10,0,1,20,40,26,2,6,72,2,3,33,0,44,11,0,5,37,3,85,15,1,0,0,0,0,8,17,4,0
27002063,2,12,24,22,13,20,18,9,2,32,3,35,26,33,0,0,3,8,8,36,0,0,15,15,5,33,2,1,6,23,3,2,29,3,3,16,12,3,21,11,...,2,29,3,1,3,26,6,2,2,33,1,0,2,7,20,7,2,2,31,1,0,12,0,19,1,1,0,9,0,33,2,0,0,0,0,0,0,2,0,0
27002950,1,70,72,109,31,52,89,57,3,123,22,133,98,116,3,3,27,48,9,138,0,2,93,29,14,121,13,6,26,91,4,4,122,9,2,69,45,4,97,21,...,1,103,29,0,4,72,56,3,5,85,42,8,3,11,52,65,10,11,111,2,1,56,0,83,4,0,8,40,1,132,9,0,0,1,1,1,7,15,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840635,2,22,31,50,5,6,45,39,1,51,4,53,46,48,3,4,9,24,3,54,1,0,18,13,23,47,3,4,12,33,2,1,37,10,2,33,16,2,45,5,...,12,48,1,4,1,33,1,20,0,45,0,9,0,20,5,26,2,6,41,0,0,20,0,29,1,0,2,17,2,37,16,1,0,1,0,0,0,7,0,0
29848873,2,8,18,18,8,9,15,9,0,24,3,25,17,25,0,0,3,8,2,25,0,0,15,2,7,23,1,1,8,13,0,0,22,1,0,12,9,0,20,3,...,5,23,1,1,0,22,0,2,0,24,0,1,0,9,5,11,0,1,22,1,0,11,0,16,0,0,0,8,0,24,1,1,0,0,0,0,0,2,0,0
29886732,2,2,4,5,0,1,4,3,0,5,0,4,5,5,0,1,1,0,1,5,0,0,4,0,0,5,0,0,0,2,0,0,2,1,0,0,2,0,3,0,...,0,4,0,0,0,2,1,0,1,2,1,0,1,2,0,1,1,0,4,0,0,3,0,4,0,0,0,2,0,4,2,0,0,0,0,0,0,0,0,0
29903726,1,19,23,32,10,16,30,16,1,33,5,39,25,28,1,2,4,10,2,38,1,0,27,8,4,36,1,2,6,20,4,4,24,4,4,19,9,4,29,1,...,10,33,0,1,4,24,1,10,3,29,2,3,3,7,7,13,11,1,33,0,2,20,1,26,0,0,1,14,3,31,9,1,0,2,0,0,2,4,0,2


In [ ]:
X_transform_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_transform_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 65.25 %
Naive Bayes: Acurácia do modelo: 61.86 %
Decision Tree: Acurácia do modelo: 44.92 %
SVM: Acurácia do modelo: 68.64 %
Logistic Regression: Acurácia do modelo: 67.8 %
KNN: Acurácia do modelo: 63.56 %
